In [11]:
import os
import cv2
import numpy as np


def generate_bbox(mask_path, image_folder):
    """
    Generates bounding box information from mask images.

    Args:
        mask_path: Path to the folder containing mask images.
        image_folder: Path to the folder containing the corresponding images (optional, for verification).

    Returns:
        A dictionary where keys are image filenames (without extension) and values are lists of bounding boxes.
        Each bounding box is a list [xmin, ymin, xmax, ymax].
        Returns an empty dictionary if no masks are found.
    """

    bbox_data = {}
    for filename in os.listdir(mask_path):
        image_info = {}
        if not filename.endswith(
            (".png", ".jpg", ".jpeg", ".bmp")
        ):  # Add other extensions as needed
            continue

        name_without_ext = os.path.splitext(filename)[0]
        mask = cv2.imread(os.path.join(mask_path, filename), cv2.IMREAD_GRAYSCALE)

        if mask is None:
            print(f"Error reading mask: {filename}")
            continue

        # Find contours in the mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        bboxes = []
        labels = []
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            bboxes.append([x, y, x + w, y + h])
            labels.append("lesion")
            

        image_info["image_name"] = filename
        image_info["image_size"] = mask.shape
        image_info["bbox"] = bboxes
        image_info["label"] = labels
        bbox_data[name_without_ext] = image_info

        # Optional: Verify bounding boxes on the image (uncomment for debugging)
        # if image_folder:
        #     image_path = os.path.join(image_folder, name_without_ext + ".jpg") # Adjust extension if needed
        #     if os.path.exists(image_path):
        #         image = cv2.imread(image_path)
        #         for bbox in bboxes:
        #             cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
        #         cv2.imshow("Image with Bounding Boxes", image)
        #         cv2.waitKey(0)
        #         cv2.destroyAllWindows()

    return bbox_data


# Example usage:
mask_folder = (
    "/media/hdd/sonwe1e/Detection/train/masks"  # Replace with your mask folder path
)
image_folder = "/media/hdd/sonwe1e/Detection/train/images"  # Replace with your image folder path (optional)

bbox_info = generate_bbox(mask_folder, image_folder)


# Print or save the bbox information (example - adapt as needed)
for filename, bboxes in bbox_info.items():
    print(f"Image: {filename}")
    for bbox in bboxes:
        print(f"  Bounding Box: {bbox}")


# You can save this information to a file (e.g., JSON or text file) for later use in your detection pipeline.
import json

with open("bbox_data.json", "w") as f:
    json.dump(bbox_info, f, indent=4)

Image: Imge_00001637
  Bounding Box: image_name
  Bounding Box: image_size
  Bounding Box: bbox
  Bounding Box: label
Image: 01.0000000165895.20012.0002.11474100368
  Bounding Box: image_name
  Bounding Box: image_size
  Bounding Box: bbox
  Bounding Box: label
Image: Imge_00006131
  Bounding Box: image_name
  Bounding Box: image_size
  Bounding Box: bbox
  Bounding Box: label
Image: 01.0000000126017.20018.0022.08584900328
  Bounding Box: image_name
  Bounding Box: image_size
  Bounding Box: bbox
  Bounding Box: label
Image: 01.0000000129524.20013.0063.14564600841
  Bounding Box: image_name
  Bounding Box: image_size
  Bounding Box: bbox
  Bounding Box: label
Image: 01.0000000129530.20013.0047.11133400908
  Bounding Box: image_name
  Bounding Box: image_size
  Bounding Box: bbox
  Bounding Box: label
Image: Imge_00010408
  Bounding Box: image_name
  Bounding Box: image_size
  Bounding Box: bbox
  Bounding Box: label
Image: Imge_00010935
  Bounding Box: image_name
  Bounding Box: image_

In [3]:
import os
import json
from PIL import Image


def yolo_to_bbox_format(yolo_annotations_path, images_path, output_json_path):
    bbox_data = {}

    for label_file in os.listdir(yolo_annotations_path):
        if label_file.endswith(".txt"):
            image_name = label_file.replace(".txt", ".png")
            image_path = os.path.join(images_path, image_name)

            # Get image size dynamically
            with Image.open(image_path) as img:
                width, height = img.size

            # Read YOLO annotation
            with open(os.path.join(yolo_annotations_path, label_file), "r") as file:
                lines = file.readlines()

            bboxes = []
            labels = []

            for line in lines:
                parts = line.strip().split()
                class_id = int(parts[0])
                cx, cy, w, h = map(float, parts[1:])

                # Convert normalized YOLO to absolute coordinates
                x_min = int((cx - w / 2) * width)
                y_min = int((cy - h / 2) * height)
                x_max = int((cx + w / 2) * width)
                y_max = int((cy + h / 2) * height)

                # Append to bounding box and label lists
                bboxes.append([x_min, y_min, x_max, y_max])
                labels.append("line")  # Using "line" as the label

            # Save in the required format
            bbox_data[image_name.split(".")[0]] = {
                "image_name": image_name,
                "image_size": [height, width],
                "bbox": bboxes,
                "label": labels,
            }

    # Write to output JSON
    with open(output_json_path, "w") as outfile:
        json.dump(bbox_data, outfile, indent=4)


# Example usage
yolo_annotations_path = "/media/hdd/sonwe1e/Detection/Data/Line/val/labels"  # Path to the YOLO annotations folder
images_path = "/media/hdd/sonwe1e/Detection/Data/Line/val/images"  # Path to the image folder
output_json_path = "/media/hdd/sonwe1e/Detection/Data/Line/val/bbox_data.json"  # Path to save the converted JSON file
yolo_to_bbox_format(yolo_annotations_path, images_path, output_json_path)
